# LangChain全面剖析之Model I/O

## 1. Model I/O介绍

### 1.1 Model I/O模块组成

Model I/O模块包括三个部分，LangChain封装了每个部分不同实现的差异
- Format：即指代Prompts Template，通过模板化来管理大模型的输入
- Predict：即指代Models，使用通用接口调用不同的大语言模型
- Parse：即指代Output部分，用来从模型的推理中提取信息，并按照预先设定好的模版来规范化输出

#### (1) Format

调用大模型时：(1) 通常会使用各种提示词工程技巧，例如Few-Shot、链式推理(CoT)等，以提高大模型的推理能力，会需要对这些工程能力进行模版化封装；(2) 另一方面，应用开发需要适应多变的用户需求和场景，提示词时不能一层不变的，也需要对提示词模版化。Format模块用来满足这类需求，通过对Prompts Template的支持，增强模型的灵活性和适用范围。

#### (2) Predict

Predict对模型推理进行封装，主要有两类[LLMs(Large Language Models)](https://python.langchain.com/v0.2/docs/integrations/llms/)和[Chat Models](https://python.langchain.com/v0.2/docs/integrations/chat/)。前者基于传统的Completion API以`文本补全`的方式进行回答，后者基于新型的Chat Completion API以`对话补全`的方式进行回答。Chat Completion API是目前的主流，但是一些老的大模型只支持Completion API，因此LangChain也对它们提供了支持。

```python
# LLMs封装形如下面例子的封装Completion API，这类API使用文本补全的方式进行回答
client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt="Say this is a test",
)
```

```python
# Chat Models用于封装Chat Completion API，这类API使用对话补全的方式进行回答。
# 它可以传入对话记录，为消息设置角色，通过role=system角色为对话设置背景信息
client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "你是一位乐于助人的AI智能小助手"},
    {"role": "user", "content": "你好，请你介绍一下你自己。"}
  ]
)
```


#### (3) Parse

大模型的输出是不稳定的，同样的输入Prompt往往会得到不同形式的输出。在自然语言交互中，不同的语言表达方式通常不会造成理解上的障碍。但在应用开发中，大模型的
输出可能是下一步逻辑处理的关键输入。因此，在这种情况下，规范化输出是必须要做的任务，以确保应用能够顺利进行后续的逻辑处理。

### 1.2 LCEL

LangChain表达式语言([LCEL](https://python.langchain.com/v0.2/docs/how_to/#langchain-expression-language-lcel))是一种声明式方法，可以轻松地将 链 组合在一起。你可以理解为就是类似shell里面管道符的开发方式。

```python
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

chain = prompt | model | StrOutputParser()
```

### 1.3 LangChain安装

In [22]:
import langchain
print(langchain.__version__)

0.1.16


In [23]:
import openai
print(openai.__version__)

1.24.0


## 2. Model I/O之模型调用

### 2.1 介绍

LangChain为了使开发者可以轻松地创建自定义链，整体采用`Runnable`协议。Runnable 协议是编程中一种常见的设计模式，用于定义可以执行的任务或行为。
在LangChain中通过构建标准接口，可以用户轻松定义自定义链并以标准方式调用它们，目前在LangChain已经集成的LLMs中，均实现了`Runnable`接口，目前支持
包括`invoke`、 `stream` 、 `batch` 、 `astream` 等方法的调用。

LangChain已经集成的大模型：

v0.2: [https://python.langchain.com/v0.2/docs/integrations/llms/](https://python.langchain.com/v0.2/docs/integrations/llms/)

v0.1: [https://python.langchain.com/docs/integrations/llms/](https://python.langchain.com/docs/integrations/llms/)

具体支持的调用方式如下所示：

| 方法    | 说明             |
|-------|----------------|
| invoke | 处理单条输入       |
| batch  | 处理批量输入 |
| stream | 流式响应         |
| ainvoke | 异步处理单条输入       |
| abatch  | 异步处理批量输入 |
| astream | 异步流式响应         |

### 2.2 不使用LangChain

下面是一段使用原生Chat Completion API编写的代码，演示如下，之后会用LangChain进行改写。

In [21]:
from openai import OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY") #把API Key预先设置在环境变量中
openai.api_base="https://api.openai.com/v1"
client = OpenAI(api_key=openai.api_key ,base_url=openai.api_base)

In [141]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "你是一个乐于助人的智能AI小助手"},
    {"role": "user", "content": "你好，请你介绍一下你自己"}
  ]
)

print(completion.choices[0].message.content)

你好！我是一个乐于助人的智能AI小助手。我被设计成可以回答各种问题、提供信息和解决问题的AI程序。无论你在需要找资料、得到建议、学习知识、做决策还是仅仅想聊天，我都会竭尽全力帮助你。我能够使用自然语言处理和大量的数据来理解你的问题，并尽可能地给出准确和有用的回答。无论你有什么需要，都请随时告诉我，我会尽力满足你的要求。


LangChain作为一个应用开发框架，需要集成各种不同的大模型，如上述OpenAI的GPT系列模型调用示例，通过Message数据输入规范，定义不同的role，即system、user和assistant来区分对话过程，但对于其他大模型，并不意味这一定会遵守这种输入输出及角色的定义，所以LangChain的做法是，因为Chat Model基于消息而不是原始文本，LangChain目前就抽象出来的消息类型有 AIMessage 、 HumanMessage 、 SystemMessage 、 FunctionMessage 和 ChatMessage ，但大多时候我们只需要处理 HumanMessage 、 AIMessage 和 SystemMessage，即：
- SystemMessage ：用于启动 AI 行为，作为输入消息序列中的第一个传入。
- HumanMessage ：表示来自与聊天模型交互的人的消息。
- AIMessage ：表示来自聊天模型的消息。这可以是文本，也可以是调用工具的请求。

### 2.3 使用LangChain的Model I/O

接下来用LangChain进行改写

In [15]:
!pip install langchain-openai

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 1.8 MB 651 kB/s eta 0:00:01
     |████████████████████████████████| 777 kB 1.1 MB/s eta 0:00:01


In [25]:
from langchain_core.messages import HumanMessage, SystemMessage

In [26]:
messages = [SystemMessage(content="你是一位乐于助人的智能小助手"),
            HumanMessage(content="你好，请你介绍一下你自己"),]

In [27]:
messages

[SystemMessage(content='你是一位乐于助人的智能小助手'), HumanMessage(content='你好，请你介绍一下你自己')]

In [43]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo",api_key=openai.api_key ,base_url=openai.api_base)

### 2.4 Model I/O提供的六种调用Chat Model的API

#### (1) invoke

用来处理单条数据

In [29]:
chat.invoke(messages)

AIMessage(content='你好！我是一个智能对话助手，可以回答各种问题，提供信息和建议。我可以帮助你解决问题、获取知识、提供娱乐和休闲建议，以及进行日常对话。无论你需要什么帮助，我都会尽力为你提供支持！', response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 42, 'total_tokens': 138}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_2f57f81c11', 'finish_reason': 'stop', 'logprobs': None}, id='run-35492861-ebe4-40c0-ac1b-9eacd865d1c8-0')

In [30]:
chat.invoke(messages).content

'你好！我是一个智能助手，可以回答各种问题，提供信息和帮助解决问题。我可以谈论各种话题，包括历史、科学、技术、健康、娱乐等等。有什么我可以帮到你的吗？'

#### (2) stream

流式响应，一边生成一边返回

In [31]:
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

你好，我是一位智能助手，可以回答各种问题并提供帮助。无论是日常生活、学习工作还是其他方面的问题，我都会尽力为您提供准确和有用的信息。有什么可以帮到您的吗？

#### (3) batch

处理批量输入

In [32]:
messages1 = [SystemMessage(content="你是一位乐于助人的智能小助手"),
 HumanMessage(content="请帮我介绍一下什么是机器学习"),]

In [33]:
messages2 = [SystemMessage(content="你是一位乐于助人的智能小助手"),
 HumanMessage(content="请帮我介绍一下什么是AIGC"),]

In [34]:
messages3 = [SystemMessage(content="你是一位乐于助人的智能小助手"),
 HumanMessage(content="请帮我介绍一下什么是大模型技术"),]

In [35]:
reponse = chat.batch([messages1,
            messages2,
            messages3,])

reponse

[AIMessage(content='机器学习是一种人工智能的分支领域，其目标是让计算机系统通过学习数据和模式识别，从而能够自动进行决策和预测。机器学习利用统计学和算法来让计算机系统从数据中学习，改进和发展自身的性能，而无需明确地进行编程。通过机器学习，计算机系统可以通过大量的数据训练和优化自己的模型，以实现各种任务，如图像识别、语音识别、自然语言处理、推荐系统等。机器学习的应用范围非常广泛，正在逐渐改变我们的日常生活和工作方式。', response_metadata={'token_usage': {'completion_tokens': 208, 'prompt_tokens': 47, 'total_tokens': 255}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_2f57f81c11', 'finish_reason': 'stop', 'logprobs': None}, id='run-e225be84-dbe8-47ec-9207-0bd5113721fa-0'),
 AIMessage(content='AIGC是Artificial Intelligence Graduate Certificate的缩写，即人工智能研究生证书。AIGC是一种专业课程，旨在培养学生在人工智能领域的技能和知识。这个证书课程通常由大学或学院提供，需要学生完成一系列的课程和项目。课程涵盖了人工智能的核心技术，包括机器学习、自然语言处理、图像识别等。获得AIGC证书的学生可以在人工智能领域的职业中取得更好的就业机会。', response_metadata={'token_usage': {'completion_tokens': 179, 'prompt_tokens': 47, 'total_tokens': 226}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ce431921-e742-498d-82e9-2b22a98bba95-0'),
 AIMessage(content='大模型技术是指利用大规模的

In [36]:
# 格式化输出

# 使用列表生成式打印每一个消息的内容
contents = [msg.content for msg in reponse]

# 打印出内容
for content in contents:
    print(content, "\n---\n")

机器学习是一种人工智能的分支领域，其目标是让计算机系统通过学习数据和模式识别，从而能够自动进行决策和预测。机器学习利用统计学和算法来让计算机系统从数据中学习，改进和发展自身的性能，而无需明确地进行编程。通过机器学习，计算机系统可以通过大量的数据训练和优化自己的模型，以实现各种任务，如图像识别、语音识别、自然语言处理、推荐系统等。机器学习的应用范围非常广泛，正在逐渐改变我们的日常生活和工作方式。 
---

AIGC是Artificial Intelligence Graduate Certificate的缩写，即人工智能研究生证书。AIGC是一种专业课程，旨在培养学生在人工智能领域的技能和知识。这个证书课程通常由大学或学院提供，需要学生完成一系列的课程和项目。课程涵盖了人工智能的核心技术，包括机器学习、自然语言处理、图像识别等。获得AIGC证书的学生可以在人工智能领域的职业中取得更好的就业机会。 
---

大模型技术是指利用大规模的数据和计算资源来训练和部署复杂、庞大的机器学习模型的技术。随着数据量的不断增加和计算能力的提升，大模型技术在人工智能领域变得越来越重要。

大模型技术通常涉及使用大规模的数据集来训练深度神经网络等复杂模型，以提高模型的准确性和泛化能力。同时，大模型技术还需要大量的计算资源来训练这些模型，包括高性能的计算机、GPU加速器等硬件设备。

大模型技术在各种领域都有广泛的应用，包括自然语言处理、计算机视觉、语音识别等。通过大模型技术，研究人员和工程师们可以构建更加复杂和强大的机器学习模型，从而实现更多领域的创新和应用。 
---



#### (4) ainvoke, abatch, astream

这三个API提供异步处理功能，分别用于异步处理单条消息、异步批量处理、异步流式响应。 

**演示同步和异步的差异**

前面演示的`invoke`本质上是一个同步调用。在这种情况下，程序会在调用返回结果之前停止执行任何后续代码。这意味着如果`invoke`操作耗时较长，它会导致程序暂时挂起，直到操作完成。我们可以通过这样一个测试代码来直观的理解同步调用：

In [37]:
import time

def call_model():
    # 模拟同步API调用
    print("开始调用模型...")
    time.sleep(5)  # 模拟调用等待
    print("模型调用完成。")

def perform_other_tasks():
    # 模拟执行其他任务
    for i in range(5):
        print(f"执行其他任务 {i + 1}")
        time.sleep(1)

def main():
    start_time = time.time()
    call_model()
    perform_other_tasks()
    end_time = time.time()
    total_time = end_time - start_time
    return f"总共耗时：{total_time}秒"

# 运行同步任务并打印完成时间
main_time = main()
main_time

开始调用模型...
模型调用完成。
执行其他任务 1
执行其他任务 2
执行其他任务 3
执行其他任务 4
执行其他任务 5


'总共耗时：10.009889841079712秒'

这段同步调用的程序先模拟了一个耗时5秒的模型调用，随后执行了五个其他任务，每个任务耗时1秒。实际的执行时间为约10.00秒。这体现了同步执行的特点：每个操作依次执行，直到当前操作完成后才开始下一个操作，从而导致总的执行时间是各个操作时间的总和。

而异步调用，允许程序在等待某些操作完成时继续执行其他任务，而不是阻塞等待。这在处理I/O操作（如网络请求、文件读写等）时特别有用，可以显著提高程序的效率和响应性。

In [38]:
import asyncio
import time

async def async_call(llm):
    await asyncio.sleep(5)  # 模拟异步操作
    print("异步调用完成")

async def perform_other_tasks():
    await asyncio.sleep(5)  # 模拟异步操作
    print("其他任务完成")

async def run_async_tasks():
    start_time = time.time()
    await asyncio.gather(
        async_call(None),  # 示例调用，替换None为模拟的LLM对象
        perform_other_tasks()
    )
    end_time = time.time()
    return f"总共耗时：{end_time - start_time}秒"

# 运行异步任务并打印完成时间
await run_async_tasks()

异步调用完成
其他任务完成


'总共耗时：5.006200075149536秒'

使用`asyncio.gather()`并行执行时，理想情况下，因为两个任务几乎同时开始，它们的执行时间将重叠。如果两个任务的执行时间相同（这里都是3秒），那么总执行时间应该接近单个任务的执行时间（3秒左右），而不是两者时间之和。

**演示Model I/O提供的异步API**

In [39]:
messages1 = [SystemMessage(content="你是一位乐于助人的智能小助手"),
 HumanMessage(content="请帮我介绍一下什么是机器学习"),]

In [40]:
reponse = await chat.ainvoke(messages1)

In [41]:
reponse.content

'当然可以！机器学习是人工智能的一个子领域，它致力于研究如何让计算机系统通过数据学习并改进性能，而无需进行显式编程。简而言之，机器学习是一种让计算机系统从数据中学习和进行预测的技术。通过训练模型，计算机可以自动识别模式、做出决策和预测结果，从而实现各种任务，如图像识别、语音识别、自然语言处理等。机器学习在各个领域都有广泛的应用，是人工智能发展的核心技术之一。'



通过上述描述，我们展示了在LangChain中使用LLMs类模型和Chat Model类模型的不同方法，其核心区别在于输入Prompt的格式。除此之外其他的工作可以直接利用统一的抽象接口，实现与模型交互的快速过程。而针对不同的模型，LangChain也提供个对应的接入方法，

其相关说明文档地址：
* v0.2: [https://python.langchain.com/v0.2/docs/integrations/chat/](https://python.langchain.com/v0.2/docs/integrations/chat/)
* v0.1: [https://python.langchain.com/docs/integrations/chat/](https://python.langchain.com/docs/integrations/chat/)

In [30]:
from langchain_community.chat_models import ChatBaichuan
from langchain_core.messages import HumanMessage

In [ ]:
chat = ChatBaichuan(
    # 这里替换成个人的有效 API KEY
    baichuan_api_key="sk-xxx",
    streaming=True,
)

In [ ]:
response = chat([HumanMessage(content="请介绍一下你自己")])
response

In [ ]:
response.content

## 3.Model I/O之Prompt Template

提示工程（Prompt Engineering）大家应该比较熟悉，这个概念是指在与大语言模型（LLMs），如GPT-3、Qwen等模型进行交互时，精心设计输入文本（即提示）的过程，以获得更精准、相关或有创造性的输出。在我们第一级学习计划中通过采用Few-Shot、Chain of Thought (CoT)等高级提示技巧，可以显著提高大模型在推理任务上的表现。

### 3.1 Prompt Template的使用

#### (1) str.format用法回顾

在LangChain的默认设置下， `PromptTemplate` 使用 Python 的 `str.format()` 方法进行模板化。

Python的`str.format()`方法是一种字符串格式化的手段，允许我们在字符串中插入变量。使用这种方法，可以创建包含占位符的字符串模板，
占位符由花括号{}标识。调用format()方法时，可以传入一个或多个参数，这些参数将被顺序替换进占位符中。str.format()提供了灵活的方式来构造字符串，
支持多种格式化选项，包括数字格式化、对齐、填充、宽度设置等。

**基本用法**

In [42]:
# 简单示例，直接替换
greeting = "Hello, {}!".format("Alice")
print(greeting)
# 输出: Hello, Alice!

Hello, Alice!


**带有位置参数的用法**

In [43]:
# 使用位置参数
info = "Name: {0}, Age: {1}".format("Bob", 30)
print(info)
# 输出: Name: Bob, Age: 30

Name: Bob, Age: 30


**带有关键字参数的用法**

In [44]:
# 使用关键字参数
info = "Name: {name}, Age: {age}".format(name="Charlie", age=25)
print(info)
# 输出: Name: Charlie, Age: 25

Name: Charlie, Age: 25


**使用字典解包的方式：**

In [45]:
# 使用字典解包
person = {"name": "David", "age": 40}
info = "Name: {name}, Age: {age}".format(**person)
print(info)
# 输出: Name: David, Age: 40

Name: David, Age: 40


#### (2) 构建PromptTemplate

在LangChain中，基本采用了Python的原生`str.format()`方法对输入数据进行格式化，这样在模型接收输入前，可以根据需要对数据进行预处理和结构
化，以此来引导大模型进行更准确的推理。

In [46]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "请给我一个关于{topic}的{type}解释。"
)

prompt= prompt_template.format(type="详细", topic="量子力学")

prompt

'请给我一个关于量子力学的详细解释。'

如上所示，可以使用`PromptTemplate`的`from_template`方法创建一个提示模板实例，这个模板包含了两个占位符：{topic} 和 {type}，这些占位
符在实际调用时可以被实际的值替换。

#### (3) 调用Chat Model

In [47]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个有帮助的AI机器人，你的名字是{name}。"),
        ("human", "你好，最近怎么样？"),
        ("ai", "我很好，谢谢！"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="小明", user_input="你叫什么名字？")

In [48]:
messages

[SystemMessage(content='你是一个有帮助的AI机器人，你的名字是小明。'),
 HumanMessage(content='你好，最近怎么样？'),
 AIMessage(content='我很好，谢谢！'),
 HumanMessage(content='你叫什么名字？')]

In [49]:
chat_template

ChatPromptTemplate(input_variables=['name', 'user_input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], template='你是一个有帮助的AI机器人，你的名字是{name}。')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='你好，最近怎么样？')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='我很好，谢谢！')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))])

&emsp;&emsp;从输出上看，其构造函数在实例化prompt_template时，主要由两个关键参数进行指定：

- input_variables：这是一个列表，包含模板中需要动态填充的变量名。这些变量名在模板字符串中以花括号（如{name}）标记。通过指定这些变量，可以在后续过程
中动态地替换这些占位符。

- template：这是定义具体提示文本的模板字符串。它可以包含静态文本和input_variables列表中指定的变量占位符。当调用format方法时，这些占位符会被实际的变
量值替换，生成最终的提示文本。

In [50]:
result = chat.invoke(messages)
print(result.content)

你好！我是ChatGPT，你可以叫我小明。有什么我可以帮助你的吗？


In [51]:
messages = chat_template.format_messages(name="张三", user_input="你要去哪里玩？")

In [52]:
result = chat.invoke(messages)
print(result.content)

作为AI助手，我并没有能力去玩耍，但是我可以为您提供旅行建议或者帮助您规划出游行程。有什么地方您想了解的吗？


### 3.2 使用PromptTemplate构造Few-Shot模版

在LangChain中，很多的功能抽象、链路抽象本质上都是在对大模型的“涌现能力”能够应用落地的一种具体实现方法，而其推理的不稳定，在不修改模型本身参数（微调）的情况下，模型涌现能力极度依赖对模型的提示过程，即对同样一个模型，不同的提示方法将获得质量完全不同的结果。最为简单的提示工程的方法就是通过输入一些类似问题和问题答案，让模型参考学习，并在同一个prompt的末尾提出新的问题，依次提升模型的推理能力。

In [53]:
prompt_Few_shot_CoT4 = 'Q：“罗杰有五个网球，他又买了两盒网球，每盒有3个网球，请问他现在总共有多少个网球？” \
                        A：“罗杰一开始有五个网球，又购买了两盒网球，每盒3个，共购买了6个网球，因此现在总共由5+6=11个网球。因此答案是11。” \
                        Q：“食堂总共有23个苹果，如果他们用掉20个苹果，然后又买了6个苹果，请问现在食堂总共有多少个苹果？” \
                        A：“食堂最初有23个苹果，用掉20个，然后又买了6个，总共有23-20+6=9个苹果，答案是9。” \
                        Q：“杂耍者可以杂耍16个球。其中一半的球是高尔夫球，其中一半的高尔夫球是蓝色的。请问总共有多少个蓝色高尔夫球？” \
                        A：“总共有16个球，其中一半是高尔夫球，也就是8个，其中一半是蓝色的，也就是4个，答案是4个。” \
                        Q：“艾米需要4分钟才能爬到滑梯顶部，她花了1分钟才滑下来，水滑梯将在15分钟后关闭，请问在关闭之前她能滑多少次？” \
                        A：'

prompt_Few_shot_CoT4

'Q：“罗杰有五个网球，他又买了两盒网球，每盒有3个网球，请问他现在总共有多少个网球？”                         A：“罗杰一开始有五个网球，又购买了两盒网球，每盒3个，共购买了6个网球，因此现在总共由5+6=11个网球。因此答案是11。”                         Q：“食堂总共有23个苹果，如果他们用掉20个苹果，然后又买了6个苹果，请问现在食堂总共有多少个苹果？”                         A：“食堂最初有23个苹果，用掉20个，然后又买了6个，总共有23-20+6=9个苹果，答案是9。”                         Q：“杂耍者可以杂耍16个球。其中一半的球是高尔夫球，其中一半的高尔夫球是蓝色的。请问总共有多少个蓝色高尔夫球？”                         A：“总共有16个球，其中一半是高尔夫球，也就是8个，其中一半是蓝色的，也就是4个，答案是4个。”                         Q：“艾米需要4分钟才能爬到滑梯顶部，她花了1分钟才滑下来，水滑梯将在15分钟后关闭，请问在关闭之前她能滑多少次？”                         A：'

In [54]:
from langchain_core.messages import HumanMessage, SystemMessage
messages = [SystemMessage(content="你是一个擅长数学推理的专家"),
 HumanMessage(content="艾米需要4分钟才能爬到滑梯顶部，她花了1分钟才滑下来，水滑梯将在15分钟后关闭，请问在关闭之前她能滑多少次？"),]

In [55]:
resonse = chat.invoke(messages)
resonse.content

'首先我们来分析一下艾米的滑梯运动：\n\n1. 艾米需要4分钟才能爬到滑梯顶部。\n2. 她每次滑下来需要1分钟。\n3. 水滑梯将在15分钟后关闭。\n\n因此，在水滑梯关闭之前，艾米能够进行的滑行次数为：  \n\n\\[ \\frac{15 \\text{分钟}}{4 \\text{分钟} (\\text{上爬}) + 1 \\text{分钟} (\\text{下滑})} = \\frac{15}{5} = 3 \\text{次}\\]\n\n所以，在水滑梯关闭之前，艾米能够滑3次。'

实际上，在使用GPT-3.5时，我们可以观察到，即使不采用Few-Shot提示，模型也能以很高的概率正确回答问题，这归功于模型本身已经非常强大的能力。

In [56]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

In [57]:
examples = [
    {"input": "罗杰有五个网球，他又买了两盒网球，每盒有3个网球，请问他现在总共有多少个网球？", 
     "output": "罗杰一开始有五个网球，又购买了两盒网球，每盒3个，共购买了6个网球，因此现在总共由5+6=11个网球。因此答案是11。"},
    
    {"input": "食堂总共有23个苹果，如果他们用掉20个苹果，然后又买了6个苹果，请问现在食堂总共有多少个苹果？", 
     "output": "食堂最初有23个苹果，用掉20个，然后又买了6个，总共有23-20+6=9个苹果，答案是9。"},
    
    {"input": "杂耍者可以杂耍16个球。其中一半的球是高尔夫球，其中一半的高尔夫球是蓝色的。请问总共有多少个蓝色高尔夫球？", 
     "output": "总共有16个球，其中一半是高尔夫球，也就是8个，其中一半是蓝色的，也就是4个，答案是4个。"},
]

In [59]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: 罗杰有五个网球，他又买了两盒网球，每盒有3个网球，请问他现在总共有多少个网球？
AI: 罗杰一开始有五个网球，又购买了两盒网球，每盒3个，共购买了6个网球，因此现在总共由5+6=11个网球。因此答案是11。
Human: 食堂总共有23个苹果，如果他们用掉20个苹果，然后又买了6个苹果，请问现在食堂总共有多少个苹果？
AI: 食堂最初有23个苹果，用掉20个，然后又买了6个，总共有23-20+6=9个苹果，答案是9。
Human: 杂耍者可以杂耍16个球。其中一半的球是高尔夫球，其中一半的高尔夫球是蓝色的。请问总共有多少个蓝色高尔夫球？
AI: 总共有16个球，其中一半是高尔夫球，也就是8个，其中一半是蓝色的，也就是4个，答案是4个。


In [60]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [61]:
final_prompt

ChatPromptTemplate(input_variables=['input'], messages=[FewShotChatMessagePromptTemplate(examples=[{'input': '罗杰有五个网球，他又买了两盒网球，每盒有3个网球，请问他现在总共有多少个网球？', 'output': '罗杰一开始有五个网球，又购买了两盒网球，每盒3个，共购买了6个网球，因此现在总共由5+6=11个网球。因此答案是11。'}, {'input': '食堂总共有23个苹果，如果他们用掉20个苹果，然后又买了6个苹果，请问现在食堂总共有多少个苹果？', 'output': '食堂最初有23个苹果，用掉20个，然后又买了6个，总共有23-20+6=9个苹果，答案是9。'}, {'input': '杂耍者可以杂耍16个球。其中一半的球是高尔夫球，其中一半的高尔夫球是蓝色的。请问总共有多少个蓝色高尔夫球？', 'output': '总共有16个球，其中一半是高尔夫球，也就是8个，其中一半是蓝色的，也就是4个，答案是4个。'}], example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], template='{output}'))])), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [62]:
chain = final_prompt | chat

response = chain.invoke({"input": "艾米需要4分钟才能爬到滑梯顶部，她花了1分钟才滑下来，水滑梯将在15分钟后关闭，请问在关闭之前她能滑多少次？"})
response.content

'在关闭之前，艾米有15分钟的时间。她每次爬到滑梯顶部需要4分钟，然后再花1分钟滑下来。所以一次循环需要5分钟。在15分钟内，她可以完成15分钟 / 5分钟 = 3次循环。因此，在关闭之前，她可以滑3次。'

### 3.3 为FewShot Template提供示例选择器

#### (1) 不使用示例选择器

In [82]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

examples = [
    # 数学推理
    {
        "question": "小明的妈妈给了他10块钱去买文具，如果一支笔3块钱，小明最多能买几支笔？",
        "answer": "小明有10块钱，每支笔3块钱，所以他最多能买3支笔，因为3*3=9，剩下1块钱不够再买一支笔。因此答案是3支。"
    },
    {
        "question": "一个篮球队有12名球员，如果教练想分成两个小组进行训练，每组需要有多少人？",
        "answer": "篮球队总共有12名球员，分成两个小组，每组有12/2=6名球员。因此每组需要有6人。"
    },
    # 逻辑推理
    {
        "question": "如果所有的猫都怕水，而Tom是一只猫，请问Tom怕水吗？",
        "answer": "根据题意，所有的猫都怕水，因此作为一只猫的Tom也会怕水。所以答案是肯定的，Tom怕水。"
    },
    {
        "question": "在夏天，如果白天温度高于30度，夜晚就会很凉爽。今天白天温度是32度，请问今晚会凉爽吗？",
        "answer": "根据题意，只要白天温度高于30度，夜晚就会很凉爽。今天白天的温度是32度，超过了30度，因此今晚会凉爽。"
    },
    # 常识问题
    {
        "question": "地球绕太阳转一圈需要多久？",
        "answer": "地球绕太阳转一圈大约需要365天，也就是一年的时间。"
    },
    {
        "question": "水的沸点是多少摄氏度？",
        "answer": "水的沸点是100摄氏度。"
    },
    # 文化常识
    {
        "question": "中国的首都是哪里？",
        "answer": "中国的首都是北京。"
    },
    {
        "question": "世界上最长的河流是哪一条？",
        "answer": "世界上最长的河流是尼罗河。"
    },
]

如上所示的示例中涵盖了数学推理、逻辑推理、常识问题以及文化常识四种不同的语义场景，并为每个场景提供了两个问题和答案。接下来将上述示例构建成提示模版

In [83]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: 小明的妈妈给了他10块钱去买文具，如果一支笔3块钱，小明最多能买几支笔？
AI: 小明有10块钱，每支笔3块钱，所以他最多能买3支笔，因为3*3=9，剩下1块钱不够再买一支笔。因此答案是3支。
Human: 一个篮球队有12名球员，如果教练想分成两个小组进行训练，每组需要有多少人？
AI: 篮球队总共有12名球员，分成两个小组，每组有12/2=6名球员。因此每组需要有6人。
Human: 如果所有的猫都怕水，而Tom是一只猫，请问Tom怕水吗？
AI: 根据题意，所有的猫都怕水，因此作为一只猫的Tom也会怕水。所以答案是肯定的，Tom怕水。
Human: 在夏天，如果白天温度高于30度，夜晚就会很凉爽。今天白天温度是32度，请问今晚会凉爽吗？
AI: 根据题意，只要白天温度高于30度，夜晚就会很凉爽。今天白天的温度是32度，超过了30度，因此今晚会凉爽。
Human: 地球绕太阳转一圈需要多久？
AI: 地球绕太阳转一圈大约需要365天，也就是一年的时间。
Human: 水的沸点是多少摄氏度？
AI: 水的沸点是100摄氏度。
Human: 中国的首都是哪里？
AI: 中国的首都是北京。
Human: 世界上最长的河流是哪一条？
AI: 世界上最长的河流是尼罗河。


In [84]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个无所不能的人，无论什么问题都可以回答。"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

final_prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='你是一个无所不能的人，无论什么问题都可以回答。')), FewShotChatMessagePromptTemplate(examples=[{'question': '小明的妈妈给了他10块钱去买文具，如果一支笔3块钱，小明最多能买几支笔？', 'answer': '小明有10块钱，每支笔3块钱，所以他最多能买3支笔，因为3*3=9，剩下1块钱不够再买一支笔。因此答案是3支。'}, {'question': '一个篮球队有12名球员，如果教练想分成两个小组进行训练，每组需要有多少人？', 'answer': '篮球队总共有12名球员，分成两个小组，每组有12/2=6名球员。因此每组需要有6人。'}, {'question': '如果所有的猫都怕水，而Tom是一只猫，请问Tom怕水吗？', 'answer': '根据题意，所有的猫都怕水，因此作为一只猫的Tom也会怕水。所以答案是肯定的，Tom怕水。'}, {'question': '在夏天，如果白天温度高于30度，夜晚就会很凉爽。今天白天温度是32度，请问今晚会凉爽吗？', 'answer': '根据题意，只要白天温度高于30度，夜晚就会很凉爽。今天白天的温度是32度，超过了30度，因此今晚会凉爽。'}, {'question': '地球绕太阳转一圈需要多久？', 'answer': '地球绕太阳转一圈大约需要365天，也就是一年的时间。'}, {'question': '水的沸点是多少摄氏度？', 'answer': '水的沸点是100摄氏度。'}, {'question': '中国的首都是哪里？', 'answer': '中国的首都是北京。'}, {'question': '世界上最长的河流是哪一条？', 'answer': '世界上最长的河流是尼罗河。'}], example_prompt=ChatPromptTemplate(input_variables=['answer', 'question'], messages=[HumanMes

In [85]:
from langchain_openai import ChatOpenAI

chain = final_prompt | chat

chain.invoke({"input": "世界上最高的山峰是哪一座"})

AIMessage(content='世界上最高的山峰是珠穆朗玛峰，位于喜马拉雅山脉。', response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 617, 'total_tokens': 654}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_2f57f81c11', 'finish_reason': 'stop', 'logprobs': None}, id='run-ab96786c-750e-46ff-b997-e8723928f443-0')

#### (2) 使用LangChain内置的示例选择器

上述过程，其实就对应了我们之前提到的问题，针对“世界上最高的山峰是哪一座？”这类问题，实际上只需输入与文化常识相关的提示就足够了。而如果要实现这一功能，就需要借助LangChain中的`example_selector`模块。在该模块中，有如下两个参数需要关注：

- example_selector ：负责为给定输入选择少数样本（以及它们返回的顺序）。它们实现了 BaseExampleSelector 接口。一个常见的例子是向量存储支持的 SemanticSimilarityExampleSelector
- example_prompt ：通过其 format_messages 方法将每个示例转换为 1 条或多条消息。一个常见的示例是将每个示例转换为一条人工消息和一条人工智能消息响应，或者一条人工消息后跟一条函数调用消息。

LangChain已经内置了多个预定义的示例选择器，每种选择器都有其特定的功能和适用场景。在这个案例中，我们先以`SemanticSimilarityExampleSelector`为例进行探索。这个选择器的目的是在给定的示例集合中选出与输入在语义上最接近的示例。主要的实现步骤如下：

1. **向量化表示**：首先，输入文本和示例集中的每个示例都会被转换成向量化的表示。通过Embedding模型将文本转换成高维空间中的点，其中语义上相似的文本会被映射到空间中相近的位置。

2. **计算语义相似度**：一旦得到了输入和示例的向量化表示，下一步是计算输入与每个示例之间的语义相似度。通过计算向量之间的距离来实现，常见的度量方式包括余弦相似度、欧氏距离等。

3. **选择最相似的示例**：基于计算出的相似度，选择一个或多个与输入最相似的示例。这个选择过程可以是简单地选取相似度最高的示例，或者根据相似度分布采取更复杂的策略，例如选择相似度高于某个阈值的所有示例。

In [11]:
! pip install chromadb

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 526 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 81.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 23.3 MB/s eta 0:00:01
     |████████████████████████████████| 283 kB 59.5 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 59.0 MB/s eta 0:00:01
     |████████████████████████████████| 41 kB 9.5 MB/s  eta 0:00:01
     |████████████████████████████████| 6.8 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 51.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 60.3 MB/s eta 0:00:01
     |████████████████████████████████| 106 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 56.8 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 78.3 MB/s 

In [86]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

使用OpenAI的Embedding模型构建向量，并存储至chromadb向量数据库中。

In [87]:
to_vectorize = [" ".join(example.values()) for example in examples]
to_vectorize

['小明的妈妈给了他10块钱去买文具，如果一支笔3块钱，小明最多能买几支笔？ 小明有10块钱，每支笔3块钱，所以他最多能买3支笔，因为3*3=9，剩下1块钱不够再买一支笔。因此答案是3支。',
 '一个篮球队有12名球员，如果教练想分成两个小组进行训练，每组需要有多少人？ 篮球队总共有12名球员，分成两个小组，每组有12/2=6名球员。因此每组需要有6人。',
 '如果所有的猫都怕水，而Tom是一只猫，请问Tom怕水吗？ 根据题意，所有的猫都怕水，因此作为一只猫的Tom也会怕水。所以答案是肯定的，Tom怕水。',
 '在夏天，如果白天温度高于30度，夜晚就会很凉爽。今天白天温度是32度，请问今晚会凉爽吗？ 根据题意，只要白天温度高于30度，夜晚就会很凉爽。今天白天的温度是32度，超过了30度，因此今晚会凉爽。',
 '地球绕太阳转一圈需要多久？ 地球绕太阳转一圈大约需要365天，也就是一年的时间。',
 '水的沸点是多少摄氏度？ 水的沸点是100摄氏度。',
 '中国的首都是哪里？ 中国的首都是北京。',
 '世界上最长的河流是哪一条？ 世界上最长的河流是尼罗河。']

In [88]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002",api_key=openai.api_key ,base_url=openai.api_base)
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

创建完vector store后，接下来需要创建 example_selector 。可以通过`k`参数指定获取多少个与输入问题最相关的示例。这里我们选择2。

In [90]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"input": "内蒙古的省会是哪里"})

[{'answer': '中国的首都是北京。', 'question': '中国的首都是哪里？'},
 {'answer': '中国的首都是北京。', 'question': '中国的首都是哪里？'}]

In [91]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"input": "罗杰有五个网球，他又买了两盒网球，每盒有3个网球，请问他现在总共有多少个网球？"})

[{'answer': '小明有10块钱，每支笔3块钱，所以他最多能买3支笔，因为3*3=9，剩下1块钱不够再买一支笔。因此答案是3支。',
  'question': '小明的妈妈给了他10块钱去买文具，如果一支笔3块钱，小明最多能买几支笔？'},
 {'answer': '小明有10块钱，每支笔3块钱，所以他最多能买3支笔，因为3*3=9，剩下1块钱不够再买一支笔。因此答案是3支。',
  'question': '小明的妈妈给了他10块钱去买文具，如果一支笔3块钱，小明最多能买几支笔？'}]

从上面两个示例我们观察到，在处理基于文化常识的查询时（例如，询问“内蒙古的省会是哪里？”），选定的few-shot模板会来源自文化常识类别。相反，当遇到需要推理的问题时，则倾向于选择我们预先定义好的数学推理类提示示例。这种动态匹配策略展示了利用语义相似性选择器在大语言模型中进行精准模板选择的能力，从而有效地应对不同类别的查询，确保模型输出的相关性和准确性。

In [92]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# 创建一个 FewShotChatMessagePromptTemplate 对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],           # 定义输入变量的列表
    example_selector=example_selector,   # 使用动态的示例选择器
    
    # 定义每一轮对话的格式化文本
    example_prompt=ChatPromptTemplate.from_messages(   
        [("human", "{question}"), ("ai", "{answer}")]
    ),
)

In [93]:
few_shot_prompt

FewShotChatMessagePromptTemplate(example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7fe9474add90>, k=2, example_keys=None, input_keys=None, vectorstore_kwargs=None), input_variables=['input'], example_prompt=ChatPromptTemplate(input_variables=['answer', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['answer'], template='{answer}'))]))

In [94]:
print(few_shot_prompt.format(input="罗杰有五个网球，他又买了两盒网球，每盒有3个网球，请问他现在总共有多少个网球？"))

Human: 小明的妈妈给了他10块钱去买文具，如果一支笔3块钱，小明最多能买几支笔？
AI: 小明有10块钱，每支笔3块钱，所以他最多能买3支笔，因为3*3=9，剩下1块钱不够再买一支笔。因此答案是3支。
Human: 小明的妈妈给了他10块钱去买文具，如果一支笔3块钱，小明最多能买几支笔？
AI: 小明有10块钱，每支笔3块钱，所以他最多能买3支笔，因为3*3=9，剩下1块钱不够再买一支笔。因此答案是3支。


In [95]:
print(few_shot_prompt.format(input="月亮每天什么时候出现"))

Human: 地球绕太阳转一圈需要多久？
AI: 地球绕太阳转一圈大约需要365天，也就是一年的时间。
Human: 地球绕太阳转一圈需要多久？
AI: 地球绕太阳转一圈大约需要365天，也就是一年的时间。


In [96]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个无所不能的人，无论什么问题都可以回答。"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [97]:
final_prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='你是一个无所不能的人，无论什么问题都可以回答。')), FewShotChatMessagePromptTemplate(example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7fe9474add90>, k=2, example_keys=None, input_keys=None, vectorstore_kwargs=None), input_variables=['input'], example_prompt=ChatPromptTemplate(input_variables=['answer', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['answer'], template='{answer}'))])), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [98]:
from langchain_openai import ChatOpenAI

chain = final_prompt | chat

chain.invoke({"input": "月亮每天什么时候出现"})

AIMessage(content='月亮每天都会出现，但出现的时间会因为月球的运动和地球的自转而有所不同。具体来说，月亮每天大约在日落后1-2小时左右出现在东方，接着会在夜间持续出现，直到天亮时从西方消失。不过，月亮的出现时间也会受到季节、地理位置等因素的影响，因此可能会有所差异。', response_metadata={'token_usage': {'completion_tokens': 130, 'prompt_tokens': 149, 'total_tokens': 279}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4dc919bc-4d5d-4619-b3b0-32f736389cbf-0')

In [99]:
from langchain_openai import ChatOpenAI

chain = final_prompt | chat

response = chain.invoke({"input": "内蒙的省会是哪座城市？"})

In [100]:
response

AIMessage(content='内蒙古的省会是呼和浩特。', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 95, 'total_tokens': 112}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_2f57f81c11', 'finish_reason': 'stop', 'logprobs': None}, id='run-31055416-0119-42d4-93a2-6aacd0fb49a0-0')

通过对`SemanticSimilarityExampleSelector`的应用，我们展示了如何动态地选择适合各种输入提示的示例模板。在LangChain框架中，示例选择器的功能和作用依赖于其具体的定义和实现。我们使用的`SemanticSimilarityExampleSelector`示例选择器，该过程涉及到示例的向量化表示、相似度计算和返回Tok这样的流程。到目前为止，LangChain已经定义了以下四种示例选择器，每种都有其独特的选择机制：

1. **Similarity**：基于输入与示例之间的语义相似度来选择示例。这种方法通过比较语义内容的接近程度来确定最相关的示例。

2. **MMR (Maximum Margin Relevance)**：根据输入与示例之间的最大边际相关性来挑选示例。这种方法旨在平衡相关性和多样性，通过选择既相关又能提供新信息的示例。

3. **Length**：依据指定长度内能够容纳的示例数量来进行选择。这个方法简单直接，特别适用于需要控制输出长度的场景。

4. **Ngram**：通过计算输入与示例之间的n-gram重叠来选择示例。这种方法重视文本表面的匹配度，适用于需要精确文本匹配的情境。

### 3.4 编写自定义示例选择器

LangChain的`ExampleSelector`模块封装了一系列较为通用的示例选择器，例如我们上一小节使用的`SemanticSimilarityExampleSelector`，它能够基于语义相似度来选择最相关示例的，已经能够满足多数提示示例使用场景的需求。然而，现实中根据不同的业务需求，可能会遇到这些通用选择器无法完全满足特定需求的情况。

在LangChain中，`Example Selector`的基本接口定义如下：

~~~python
class BaseExampleSelector(ABC):
    """用于选择要包含在提示中的示例的接口。"""

    @abstractmethod
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
    """根据输入选择使用哪些示例。"""

    @abstractmethod
    def add_example(self, example: Dict[str, str]) -> Any:
        """向存储中添加新的示例。"""
~~~

ABC，全称为“Abstract Base Class”（抽象基类），是Python中abc模块的一部分。在 Python 中，抽象基类用于定义其他类必须遵循的基本接口或蓝图，但不能直接实例化。其主要目的是为了提供一种形式化的方式来定义和检查子类的接口。抽象基类中，可以定义抽象方法，它没有实现（也就是说，它没有方法体）。任何继承该抽象基类的子类都必须提供这些抽象方法的实现。

从上述基本接口来看，它需要定义的唯一方法是 `select_examples` 方法，其接受输入变量，然后返回示例列表。如何选择这些示例取决于每个具体的实现，也就是我们自定义的逻辑。

为了演示示例选择器的自定义过程，我们设计这样一个简单的场景：聊天机器人的回答选择器。在这个场景中，聊天机器人需要根据用户的输入从一个预设的回答库中选择最合适的回答。这个预设库包含了多个输入-回答对，机器人的任务是找到与用户输入在长度上最接近的问题，然后返回相应的预设回答。通过这种方法来可以帮助机器人处理未知或罕见的用户输入，通过匹配相近长度的问题来给出一个看似合适的回答，增加用户满意度。

In [116]:
examples = [
    {"input": "你好吗？", "output": "我很好，谢谢！你呢？"},
    {"input": "你是谁？", "output": "我是一个聊天机器人。"},
    {"input": "你能做什么？", "output": "我可以回答简单的问题，比如现在的时间或天气。"},
    {"input": "现在几点了？", "output": "抱歉，我无法提供实时信息。"},
    {"input": "你喜欢音乐吗？", "output": "我不能听音乐，但我可以帮你找到音乐信息。"},
    {"input": "告诉我一些关于中国的事情。", "output": "中国是一个拥有悠久历史和丰富文化的国家。"},
    {"input": "最近有什么好玩的电影吗？", "output": "我不太清楚当前的电影信息，但我推荐你查看电影推荐网站。"},
    {"input": "你能帮我学习编程吗？", "output": "当然，我可以提供一些学习资源和编程练习。"}
]

这个examples列表包含了几个不同长度的问题及其对应的答案。根据聊天机器人的回答选择器的需求设定，我们设计的示例选择器功能就应该是：当用户输入一段文本时，自定义示例选择器的的`select_examples`方法会根据输入的长度选择一个最接近的问题，并返回那个问题的答案。这样，即使用户的问题没有直接出现在预设的问题列表中，聊天机器人也能提供一个相关的回答。

In [117]:
from langchain_core.example_selectors.base import BaseExampleSelector

class ChatbotExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        # examples是一个列表，包含多个字典，每个字典都有'input'和'output'键
        self.examples = examples

    def add_example(self, example):
        # 向examples列表添加一个输入-输出对
        self.examples.append(example)

    def select_examples(self, input_variables):
        # 此方法找到与用户输入长度最接近的示例，并返回相应的输出
        new_word = input_variables["input"]
        new_word_length = len(new_word)

        best_match = None
        ## 声明一个无穷大的变量
        smallest_diff = float("inf")

        for example in self.examples:
            current_diff = abs(len(example["input"]) - new_word_length)

            if current_diff < smallest_diff:
                smallest_diff = current_diff
                best_match = example

        # 如果找到了最佳匹配项，返回相应的输出；否则，返回None
        return [best_match] if best_match else []
    


In [118]:
example_selector = ChatbotExampleSelector(examples)
example_selector

In [119]:
example_selector.select_examples({"input": "你好呀。"})

[{'input': '你好吗？', 'output': '我很好，谢谢！你呢？'}]

In [120]:
example_selector.select_examples({"input": "我特别的喜欢打篮球"})

[{'input': '你能帮我学习编程吗？', 'output': '当然，我可以提供一些学习资源和编程练习。'}]

In [121]:
example_selector.select_examples({"input": "今天的天气很好，能推荐一个好玩的去处吗？"})

[{'input': '告诉我一些关于中国的事情。', 'output': '中国是一个拥有悠久历史和丰富文化的国家。'}]

In [122]:
example_selector.select_examples({"input": "今天的天气很好，非常适合春游，能帮我推荐一个适合全家人出游的好去处吗？"})

[{'input': '告诉我一些关于中国的事情。', 'output': '中国是一个拥有悠久历史和丰富文化的国家。'}]

In [123]:
example_selector.add_example({"input": "春天到了，大家都喜欢出去春游，但是很多地方并不是很好，请问有推荐码？", "output": "如果你喜欢春天春游的话，你可以去一些国家公园，景色非常好。"})

In [124]:
example_selector.select_examples({"input": "今天的天气很好，非常适合春游，能帮我推荐一个适合全家人出游的好去处吗？"})

[{'input': '春天到了，大家都喜欢出去春游，但是很多地方并不是很好，请问有推荐码？',
  'output': '如果你喜欢春天春游的话，你可以去一些国家公园，景色非常好。'}]

这里就可以匹配到最新添加的提示模版了。而对接大模型推理过程就和常规的使用方式无异。首先在对话模版中接入`ChatbotExampleSelector`示例选择器，代码如下：

In [125]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# 创建一个 FewShotChatMessagePromptTemplate 对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],           # 定义输入变量的列表
    example_selector=example_selector,   # 使用动态的示例选择器
    
    # 定义每一轮对话的格式化文本
    example_prompt=ChatPromptTemplate.from_messages(   
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

In [126]:
print(few_shot_prompt.format(input="今天的天气很好，非常适合春游，能帮我推荐一个适合全家人出游的好去处吗？"))

Human: 春天到了，大家都喜欢出去春游，但是很多地方并不是很好，请问有推荐码？
AI: 如果你喜欢春天春游的话，你可以去一些国家公园，景色非常好。


In [127]:
print(few_shot_prompt.format(input="你好呀。"))

Human: 你好吗？
AI: 我很好，谢谢！你呢？


In [128]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个无所不能的人，无论什么问题都可以回答。"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [129]:
final_prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='你是一个无所不能的人，无论什么问题都可以回答。')), FewShotChatMessagePromptTemplate(example_selector=<__main__.ChatbotExampleSelector object at 0x7fe92d701d90>, input_variables=['input'], example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], template='{output}'))])), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [130]:
from langchain_openai import ChatOpenAI

chain = final_prompt | chat

chain.invoke({"input": "你好呀"})

AIMessage(content='你好！有什么问题我可以帮你解答吗？', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 67, 'total_tokens': 86}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bfe754fc-4b07-42b1-9b47-993526bb8362-0')

In [131]:
from langchain_openai import ChatOpenAI

chain = final_prompt | chat

chain.invoke({"input": "你是谁？"})

AIMessage(content='我是一个人工智能语言模型，被称为AI助手。我被设计成可以回答各种问题和提供帮助。', response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 68, 'total_tokens': 111}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2c0d3d3e-1030-43b8-8110-fdf5ef3e319b-0')

In [132]:
from langchain_openai import ChatOpenAI

chain = final_prompt | chat

response = chain.invoke({"input": "今天的天气很好，非常适合春游，能帮我推荐一个适合全家人出游的好去处吗？"})
response.content

'当然可以，以下是一些适合全家人春季出游的地方：\n\n1. 桂林漓江\n\n漓江是桂林的一条著名河流，以其秀丽的山水风光而著名。你可以选择坐游船游览漓江，欣赏河流两岸的奇峰异石和青山绿水，也可以选择徒步穿越漓江的峡谷，感受一下自然的魅力。\n\n2. 黄山\n\n黄山是中国著名的山脉之一，以其奇峰、云海、日出、松柏等景观而著名。你可以选择在山脚下的村庄住宿，或者在山上的酒店住宿，欣赏黄山的自然美景。\n\n3. 长城\n\n长城是中国最著名的旅游景点之一，也是世界上最长的城墙之一。你可以选择在长城上徒步，欣赏长城的壮丽景色，或者在长城下的村庄中住宿，体验当地的文化和风味美食。\n\n希望以上推荐对你有帮助，祝你和你的家人春季出游愉快！'

## 4.  Model I/O之Output Parsers

### 4.1 介绍

Output Parsers，即输出解析器，这个概念非常好理解，就是负责获取大模型的输出并将其转换为更合适的格式。这在应用开发中及其重要。在大多数复杂应用场景中，处理逻辑往往环环相扣，执行某项业务逻辑可能需要多次调用大模型，其中上一次的调用结果将被用于指导下一次调用的逻辑。在这种情况下，结构化的信息会比纯文本又有价值，同时这也是输出解析器的价值所在。

LangChain构造的输出解释器必须实现两个主要方法：
- Get format instructions：该方法会返回一个字符串，其中包含有关如何格式化语言模型输出的指令。
- Parse：该方法会接收字符串，并将其解析为某种结构

目前已经支持的解析格式已经包括Json、Xml、Csv以及OpenAI的Tools和Functions等多种格式，具体可看：

v0.2: [https://python.langchain.com/v0.2/docs/concepts/#output-parsers](https://python.langchain.com/v0.2/docs/concepts/#output-parsers) 
, [https://python.langchain.com/v0.2/docs/how_to/#output-parsers](https://python.langchain.com/v0.2/docs/how_to/#output-parsers) 

v0.1: [https://python.langchain.com/docs/modules/model_io/output_parsers/](https://python.langchain.com/docs/modules/model_io/output_parsers/) 

### 4.2 常见Output Parser

Examples (v0.1) 

* CSV: [https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/csv/](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/csv/) 
* DataTime: [https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/datetime/](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/datetime/) 
* Enum: [https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/enum/](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/enum/) 
* Json: [https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/json/](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/json/) 
* PandasDataFrame: [https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/pandas_dataframe/](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/pandas_dataframe/) 
* Pydantic: [https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/pydantic/](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/pydantic/)
* ...

### 4.3 例子

#### (1) Datetime parser

In [142]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

output_parser = DatetimeOutputParser()

In [143]:
output_parser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1340-06-19T16:20:36.785423Z, 1449-09-11T02:24:41.532562Z, 1581-04-20T04:18:49.834816Z\n\nReturn ONLY this string, no other words!"

In [144]:
template = """用户发起的提问:

{question}

{format_instructions}"""


In [145]:

prompt = PromptTemplate.from_template(
    template,
    # 预定义的变量，这里我们传入格式化指令
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)


In [146]:
prompt

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1763-04-12T15:47:45.056396Z, 161-02-08T17:48:02.682900Z, 526-10-31T17:39:53.463104Z\n\nReturn ONLY this string, no other words!"}, template='用户发起的提问:\n\n{question}\n\n{format_instructions}')

In [149]:

chain = prompt | chat | output_parser

In [150]:
output = chain.invoke({"question": "你好，请问你叫什么？"})
output


datetime.datetime(2022, 5, 30, 12, 34, 56, 789012)

In [151]:
print(output)

2022-05-30 12:34:56.789012


#### (2) JSON parser

In [153]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [154]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [155]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | chat | parser

chain.invoke({"query": joke_query})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

### 4.4 理解Output Parser的实现

查看这些Parser的源代码，可以看到它的实现原理。以JsonOutputParser为例

[https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/json.py](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/json.py)

```python
class JsonOutputParser(BaseCumulativeTransformOutputParser[Any]):
    # ...    
    pydantic_object: Optional[Type[TBaseModel]] = None  # type: ignore

    def _get_schema(self, pydantic_object: Type[TBaseModel]) -> dict[str, Any]:
        if PYDANTIC_MAJOR_VERSION == 2:
            if issubclass(pydantic_object, pydantic.BaseModel):
                return pydantic_object.model_json_schema()
            elif issubclass(pydantic_object, pydantic.v1.BaseModel):
                return pydantic_object.schema()
        return pydantic_object.schema()

    def get_format_instructions(self) -> str:
        schema = {k: v for k, v in self._get_schema(self.pydantic_object).items()}
        # ...
        schema_str = json.dumps(reduced_schema)
        return JSON_FORMAT_INSTRUCTIONS.format(schema=schema_str)
```

[https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/format_instructions.py](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/format_instructions.py)

~~~python
JSON_FORMAT_INSTRUCTIONS = """The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {{"properties": {{"foo": {{"title": "Foo", "description": "a list of strings", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}
the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of the schema. The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Here is the output schema:
```
{schema}
```"""
~~~

LangChain的源代码：[https://github.com/langchain-ai/langchain](https://github.com/langchain-ai/langchain) 

## 5. Ollama下载和部署本地私有模型

Ollama是在Github上的一个开源项目，其项目定位是：**一个本地运行大模型的集成框架**，目前主要针对主流的LLaMA架构的开源大模型设计，通过将模型权重、配置文件和必要数据封装进由`Modelfile`定义的包中，从而实现大模型的下载、启动和本地运行的自动化部署及推理流程。此外，Ollama内置了一系列针对大模型运行和推理的优化策略，目前作为一个非常热门的大模型托管平台，已被包括LangChain、Taskweaver等在内的多个热门项目高度集成。




Ollama官方地址：[https://ollama.com/](https://ollama.com/)

Ollama Github开源地址：[https://github.com/ollama/ollama](https://github.com/ollama/ollama)

Ollama项目支持跨平台部署，目前已兼容Mac、Linux和Windows操作系统。特别地对于Windows用户提供了非常直观的预览版，包括了内置的GPU加速功能、访问完整模型库的能力，以及对OpenAI的兼容性在内的Ollama API，使其对Windows用户尤为友好。而无论在使用哪个操作系统中，Ollama项目的安装过程都设计得非常简单。示例中以Linux版本为例进行详细介绍。对于其他操作系统版本的安装，可以通过上面的链接，根据自己的实际情况进行安装体验

一键安装的过程极为简便，仅需通过执行以下命令行即可自动化完成

下面是在AutoDL租用的Linux上安装的例子，需要在AutoDL的Portal上为这台机器开启“学术加速”功能，方便它能够畅通快速下载Lib和模型

```bash
sudo apt-get update
sudo apt-get install pciutils lshw #AutoDL提供的Linux缺少这几个Lib
    
curl -fsSL https://ollama.com/install.sh | sh
```

这行命令的目的是从`https://ollama.com/` 网站读取 `install.sh` 脚本，并立即通过 `sh` 执行该脚本，在安装过程中会包含以下几个主要的操作：
1. 检查当前服务器的基础环境，如系统版本等；
2. 下载Ollama的二进制文件；
3. 配置系统服务，包括创建用户和用户组，添加Ollama的配置信息；
4. 启动Ollama服务；

在Ollama官网上，可以找到拉去大模型的命令。为了节省下载时间和比较小的GPU，选择一个比较小的qwen:0.5b-chat模型试验，本试验在AutoDL上租用了GPU是NVIDA GeForce RTX 4090的机器，使用nvidea smi命令查看GPU配置）

模型页面：[https://ollama.com/library/qwen:0.5b-chat](https://ollama.com/library/qwen:0.5b-chat) 

在前面安装了ollma的机器上，先启动ollama

```bash
ollama #查看命令行参数
ollama serve #启动ollama
```

然后再执行从模型页面上得到的命令

```bash
ollama pull qwen:0.5b-chat # 获取模型
ollama run qwen:0.5b-chat  # 加载模型
```

大模型就在本地加载好了

## 6. LangChain调用私有模型

### 6.1 阅读文档

先到LangChain Guides中找到相关的How To文档

LangChain Guides：
* v0.2: [https://python.langchain.com/v0.2/docs/how_to/](https://python.langchain.com/v0.2/docs/how_to/) 
* v0.1: [https://python.langchain.com/v0.1/docs/guides/development/local_llms/](https://python.langchain.com/v0.1/docs/guides/development/local_llms/) 

调用本地大模型的How To文档
* v0.2: [https://python.langchain.com/v0.2/docs/how_to/local_llms/](https://python.langchain.com/v0.2/docs/how_to/local_llms/) 
* v0.1: [https://python.langchain.com/v0.1/docs/guides/development/local_llms/](https://python.langchain.com/v0.1/docs/guides/development/local_llms/) 

根据这些文档，就知道如何使用LangChain访问本地大模型了

### 6.2 验证本地私有模型可用

In [2]:
from langchain_community.chat_models import ChatOllama

In [3]:
ollama_llm = ChatOllama(model="qwen:0.5b-chat")

In [4]:
from langchain_core.messages import HumanMessage

messages = [
    HumanMessage(
        content="你好，请你介绍一下你自己",
    )
]

In [5]:
chat_model_response = ollama_llm.invoke(messages)

chat_model_response

AIMessage(content='您好！我是来自阿里云的超大规模语言模型“通义千问”。我拥有多项世界顶级的自然语言处理能力。我的目标是帮助用户获取准确、有用的信息，解决实际问题。', response_metadata={'model': 'qwen:0.5b-chat', 'created_at': '2024-05-10T04:28:56.912789971Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 4045618079, 'load_duration': 3578536348, 'prompt_eval_count': 13, 'prompt_eval_duration': 22027000, 'eval_count': 45, 'eval_duration': 313668000}, id='run-a81827f1-1827-42d9-b816-c176fca27a65-0')

In [6]:
chat_model_response.content

'您好！我是来自阿里云的超大规模语言模型“通义千问”。我拥有多项世界顶级的自然语言处理能力。我的目标是帮助用户获取准确、有用的信息，解决实际问题。'

In [7]:
messages = [
    HumanMessage(
        content="请问什么是机器学习?",
    )
]

In [8]:
chat_model_response = ollama_llm.invoke(messages)

chat_model_response

AIMessage(content='机器学习是一种人工智能技术，它的目标是通过训练数据来自动提取特征并进行分类或预测。机器学习主要应用于计算机视觉、自然语言处理等领域。', response_metadata={'model': 'qwen:0.5b-chat', 'created_at': '2024-05-10T04:29:40.321914775Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 413635745, 'load_duration': 2588838, 'prompt_eval_count': 10, 'prompt_eval_duration': 24992000, 'eval_count': 35, 'eval_duration': 248933000}, id='run-bb6283cb-8db9-4716-91dd-ef2f326ff0cc-0')

In [9]:
chat_model_response.content

'机器学习是一种人工智能技术，它的目标是通过训练数据来自动提取特征并进行分类或预测。机器学习主要应用于计算机视觉、自然语言处理等领域。'

### 6.3 使用LangChain调用本地私有模型

In [10]:
from langchain.prompts.chat import ChatPromptTemplate

# 构建模版
template = "你是一个有用的助手，可以将{input_language}翻译成{output_language}。"
human_template = "{text}"

# 生成对话形式的聊天信息格式
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_prompt

ChatPromptTemplate(input_variables=['input_language', 'output_language', 'text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input_language', 'output_language'], template='你是一个有用的助手，可以将{input_language}翻译成{output_language}。')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])

In [11]:
# 格式化变量输入
messages = chat_prompt.format_messages(input_language="中文", output_language="英语", text="我爱编程")
messages

[SystemMessage(content='你是一个有用的助手，可以将中文翻译成英语。'), HumanMessage(content='我爱编程')]

In [12]:
from langchain_community.chat_models import ChatOllama

# 实例化Ollama启动的模型
ollama_llm = ChatOllama(model="qwen:0.5b-chat")

# 执行推理
result = ollama_llm.invoke(messages)
print(result.content)

I love programming.


In [13]:
# 格式化变量输入
messages = chat_prompt.format_messages(input_language="中文", output_language="英语", text="我喜欢打篮球")
messages

[SystemMessage(content='你是一个有用的助手，可以将中文翻译成英语。'),
 HumanMessage(content='我喜欢打篮球')]

In [14]:
# 执行推理
result = ollama_llm.invoke(messages)
print(result.content)

I enjoy playing basketball.


## 7. LangChain调用外部函数

### 7.1 大模型调用外部函数

#### (1) 外部函数编写和封装

在[https://home.openweathermap.org/](https://home.openweathermap.org/)上申请一个API Key，然后就可以调用它的Rest API获取当前的天气。

In [16]:
import numpy as np
import pandas as pd

import json
import io
import inspect
import requests

In [17]:
open_weather_key = "5c939a7cc59eb8696f4cd77bf75c5a9a"

In [18]:
import requests

# Step 1.构建请求
url = "https://api.openweathermap.org/data/2.5/weather"

# Step 2.设置查询参数
params = {
    "q": "Beijing",               # 查询北京实时天气
    "appid": open_weather_key,    # 注意：这里需要替换为实际的 OpenWeather API key
    "units": "metric",            # 使用摄氏度而不是华氏度
    "lang":"zh_cn"                # 输出语言为简体中文
}

# Step 3.发送GET请求
response = requests.get(url, params=params)

# Step 4.解析响应
data = response.json()

In [19]:
data

{'coord': {'lon': 116.3972, 'lat': 39.9075},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': '阴，多云',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 25.94,
  'feels_like': 25.29,
  'temp_min': 25.94,
  'temp_max': 25.94,
  'pressure': 1001,
  'humidity': 27,
  'sea_level': 1001,
  'grnd_level': 995},
 'visibility': 10000,
 'wind': {'speed': 3.85, 'deg': 222, 'gust': 6.13},
 'clouds': {'all': 94},
 'dt': 1715316111,
 'sys': {'type': 1,
  'id': 9609,
  'country': 'CN',
  'sunrise': 1715288663,
  'sunset': 1715339838},
 'timezone': 28800,
 'id': 1816670,
 'name': 'Beijing',
 'cod': 200}

In [20]:
# 即时温度最高、最低气温
data['main']['temp_min'], data['main']['temp_max']

(25.94, 25.94)

In [21]:
# 天气状况
data['weather'][0]['description']

'阴，多云'

将获取天气的逻辑封装在一个函数中，就是我们要使用的外部函数

In [22]:
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

In [23]:
get_weather('ShangHai')

'{"coord": {"lon": 121.4581, "lat": 31.2222}, "weather": [{"id": 800, "main": "Clear", "description": "\\u6674", "icon": "01d"}], "base": "stations", "main": {"temp": 25.86, "feels_like": 25.91, "temp_min": 23.93, "temp_max": 26.63, "pressure": 1015, "humidity": 54}, "visibility": 10000, "wind": {"speed": 7, "deg": 150}, "clouds": {"all": 0}, "dt": 1715316082, "sys": {"type": 2, "id": 145096, "country": "CN", "sunrise": 1715288518, "sunset": 1715337554}, "timezone": 28800, "id": 1796236, "name": "Shanghai", "cod": 200}'

In [24]:
get_weather('Beijing')

'{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\\u9634\\uff0c\\u591a\\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 25.94, "feels_like": 25.29, "temp_min": 25.94, "temp_max": 25.94, "pressure": 1001, "humidity": 27, "sea_level": 1001, "grnd_level": 995}, "visibility": 10000, "wind": {"speed": 3.85, "deg": 222, "gust": 6.13}, "clouds": {"all": 94}, "dt": 1715316111, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715288663, "sunset": 1715339838}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}'

#### (2) 使用Chat Completion API调用外部函数

如果不使用LangChain，外部函数调用过程就是以前提高过的两阶段调用

步骤如下
1. 为外部函数生成Tool Specification，开发阶段借助大模型生成，产品部署前将Tool Specification写成固定配置，提高稳定性。
2. 初始化Open AI的原生client，调用它的Chat Completion API传入用户提问，同时传入先前生成的Tool Specification
3. 第一阶段：大模型会根据用户提问以及Tool Specification的内容判断是否需要执行外部函数，如果不需要则直接以Text的形式返回文本，如果需要则根据固定的Schema返回Json，告诉调用方需要调用哪些Tool以及为这这些Tool生成了怎样的参数
4. 第二阶段：如果需要调用Tool，调用方负责调用相对应地外部函数，然后将外部函数的返回值，把返回值与用户的提问一起交给大模型，生成最终的回答

In [15]:
import openai
import os
import numpy as np
import pandas as pd
import json
import io
from openai import OpenAI
import inspect

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base="https://api.openai.com/v1"


client = OpenAI(api_key=openai.api_key ,base_url=openai.api_base)

In [25]:
def auto_functions(functions_list):
    """
    Chat模型的functions参数编写函数
    :param functions_list: 包含一个或者多个函数对象的列表；
    :return：满足Chat模型functions参数要求的functions对象
    """
    def functions_generate(functions_list):
        # 创建空列表，用于保存每个函数的描述字典
        functions = []
        # 对每个外部函数进行循环
        for function in functions_list:
            # 读取函数对象的函数说明
            function_description = inspect.getdoc(function)
            # 读取函数的函数名字符串
            function_name = function.__name__

            system_prompt = '以下是某的函数说明：%s' % function_description
            user_prompt = '根据这个函数的函数说明，请帮我创建一个JSON格式的字典，这个字典有如下5点要求：\
                           1.字典总共有三个键值对；\
                           2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
                           3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
                           4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
                           5.输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_name

            response = client.chat.completions.create(
                              model="gpt-3.5-turbo",
                              messages=[
                                {"role": "system", "content": system_prompt},
                                {"role": "user", "content": user_prompt}
                              ]
                            )
            json_function_description=json.loads(response.choices[0].message.content.replace("```","").replace("json",""))
            json_str={"type": "function","function":json_function_description}
            functions.append(json_str)
        return functions
    ## 最大可以尝试4次
    max_attempts = 4
    attempts = 0

    while attempts < max_attempts:
        try:
            functions = functions_generate(functions_list)
            break  # 如果代码成功执行，跳出循环
        except Exception as e:
            attempts += 1  # 增加尝试次数
            print("发生错误：", e)
            if attempts == max_attempts:
                print("已达到最大尝试次数，程序终止。")
                raise  # 重新引发最后一个异常
            else:
                print("正在重新运行...")
    return functions

In [26]:
functions_list = [get_weather]
functions_list

[<function __main__.get_weather(loc)>]

In [27]:
functions = auto_functions(functions_list)
functions

[{'type': 'function',
  'function': {'name': 'get_weather',
   'description': '查询即时天气函数',
   'parameters': {'loc': {'type': 'string',
     'description': "必要参数，用于表示查询天气的具体城市名称。中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'。"}}}}]

In [28]:
def run_conversation(messages, functions_list=None, model="gpt-3.5-turbo"):
    """
    能够自动执行外部函数调用的对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为gpt-3.5-turbo
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response.choices[0].message
        final_response = response_message.content
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        tools = auto_functions(functions_list)

        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}

        # 第一次调用大模型
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        tools=tools,
                        tool_choice="auto", )
        response_message = response.choices[0].message


        tool_calls = response_message.tool_calls

        if tool_calls:

            messages.append(response_message) 
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                ## 真正执行外部函数的就是这儿的代码
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                ) 
            ## 第二次调用模型
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
            ) 
            # 获取最终结果
            final_response = second_response.choices[0].message.content
        else:
            final_response = response_message.content
                
    return final_response

In [29]:
functions_list = [get_weather]

In [30]:
messages = [{"role": "user", "content": '今天北京的天气如何？'}]

run_conversation(messages=messages, 
                 functions_list=functions_list, 
                )

'北京今天的天气是多云，气温约为26.94摄氏度，湿度为27%。风速约为3.85米/秒，风向为222度。整体来看是一个舒适的天气。'

In [31]:
def chat_with_model(functions_list=None, 
                    prompt="你好呀", 
                    model="gpt-3.5-turbo",
                    system_message=[{"role": "system", "content": "你是以为乐于助人的助手。"}]):
    
    messages = system_message
    messages.append({"role": "user", "content": prompt})
    
    while True:           
        answer = run_conversation(messages=messages, 
                                    functions_list=functions_list, 
                                    model=model)
        
        
        print(f"模型回答: {answer}")

        # 询问用户是否还有其他问题
        user_input = input("您还有其他问题吗？(输入退出以结束对话): ")
        if user_input == "退出":
            break

        # 记录用户回答
        messages.append({"role": "user", "content": user_input})

In [35]:
chat_with_model(functions_list, prompt="你好")

模型回答: 你好！北京今天的天气预报显示晴天，最高温度约为20摄氏度，最低温度约为8摄氏度。


您还有其他问题吗？(输入退出以结束对话):  杭州今天的天气如何？


模型回答: 根据最新的天气数据，北京今天的天气为多云，气温为26.94摄氏度，相对湿度为27%。而杭州今天的天气也是多云，气温为30.5摄氏度，相对湿度为45%。希望这个信息对你有帮助！


您还有其他问题吗？(输入退出以结束对话):  介绍一下你自己？


BadRequestError: Error code: 400 - {'error': {'message': 'Invalid schema for function \'get_weather\': schema must be a JSON Schema of \'type: "object"\', got \'type: "None"\'. (request id: 20240510125413744519940nY7mYLbU) (request id: 20240510125413509143154EAcrgXet) (request id: 20240510125358938800532vDAScyrY) (request id: 20240510125413152448211KotLCiyL)', 'type': 'invalid_request_error', 'param': '', 'code': None}}

因为Tool Specification是使用大模型自动生成的，存在一定的出错概率，例子如上，因此在生产部署时，会将Tool Specification存储为固定的配置

### 7.2 LangChain调用外部函数

#### (1) 用`@tool`装饰器装饰外部函数

导入依赖

In [36]:
from langchain.tools import BaseTool, StructuredTool, tool

用`@tool`装饰器来装饰外部函数，被装饰后它就变成了一个LangChain的LLM Tool，能够获取name、description、args等属性，还能够通过invoke方法来调用

In [37]:
@tool
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

测试一下这个函数

In [38]:
get_weather

StructuredTool(name='get_weather', description="get_weather(loc) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息", args_schema=<class 'pydantic.v1.main.get_weatherSchema'>, func=<function get_weather at 0x7ff8c8638160>)

如上所示，使用`@tool`装饰器可以直接将`get_weather`函数转换成工具，这个工具可以用来执行调用，并处理返回的结果。同时，可以支持一些内部方法的调用。

In [39]:
print(get_weather.name)
print(get_weather.description)
print(get_weather.args)

get_weather
get_weather(loc) - 查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
{'loc': {'title': 'Loc'}}


In [40]:
get_weather.invoke({"loc": "Beijing"})

'{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\\u9634\\uff0c\\u591a\\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 26.94, "feels_like": 26.27, "temp_min": 26.94, "temp_max": 26.94, "pressure": 1001, "humidity": 27, "sea_level": 1001, "grnd_level": 995}, "visibility": 10000, "wind": {"speed": 3.85, "deg": 222, "gust": 6.13}, "clouds": {"all": 94}, "dt": 1715317603, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715288663, "sunset": 1715339838}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}'

#### (2) 将tool绑定到LangChain的Chat Model上

Chat Model就是前面说的，LangChain对基于Chat Completion API的大模型的封装

chat变量在前面已经初始化

```python
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo",api_key=openai.api_key ,base_url=openai.api_base)
```

绑定代码如下

In [44]:
llm_with_tools = chat.bind_tools([get_weather])
llm_with_tools

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7ff8c8180310>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7ff8c8184a60>, openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1', openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': "get_weather(loc) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息", 'parameters': {'type': 'object', 'properties': {'loc': {}}, 'required': ['loc']}}}]})

In [45]:
llm_with_tools.invoke("北京的天气怎么样？")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KhcAfPhgY6adqs0am5WYqy16', 'function': {'arguments': '{"loc":"Beijing"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 198, 'total_tokens': 213}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_2f57f81c11', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-64d560b0-9485-4c6f-a7e3-a47fee73fa49-0', tool_calls=[{'name': 'get_weather', 'args': {'loc': 'Beijing'}, 'id': 'call_KhcAfPhgY6adqs0am5WYqy16'}])

In [46]:
llm_with_tools.kwargs["tools"]

[{'type': 'function',
  'function': {'name': 'get_weather',
   'description': "get_weather(loc) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息",
   'parameters': {'type': 'object',
    'properties': {'loc': {}},
    'required': ['loc']}}}]

返回的数据中存在一个`additional_kwargs` 属性，这个属性在LangChain中是用来传递有关Messages的附加信息，主要用于特定于提供者而非通用的输入参数。这个例子中，它被用来传递Function Calling的信息（即大模型认为需要调用哪个tool，给这个tool的参数是什么）

这个例子本质是实现的是Function Calling中第一轮对话的功能，即根据用户提问判断是否需要使用tool，使用哪些，参数是什么

#### (3) 解析大模型返回的Function Calling信息

大模型返回的Function Calling信息，并不能直接用于本地函数的执行，需要使用一个Output Parser进行解析和转换。LangChain提供了一个`JsonOutputKeyToolsParser`类用于该场景，它继承了` JsonOutputToolsParser `类构建工具调用模型，它会将 OpenAI 函数调用响应转换为 {"type": "TOOL_NAME", "args": {...}} 字典列表调用和调用它们的参数（本质上与我们在前面手动提取的逻辑一致），一个简单的示例如下。

In [47]:
from langchain.output_parsers import JsonOutputKeyToolsParser

In [48]:
chain = llm_with_tools | JsonOutputKeyToolsParser(key_name='get_weather', first_tool_only=True)
chain.invoke("杭州的天气怎么样？")

{'loc': 'Hangzhou'}

In [49]:
chain = llm_with_tools | JsonOutputKeyToolsParser(key_name='get_weather', first_tool_only=True)
chain.invoke("今天北京的天气好吗？")

{'loc': 'Beijing'}

#### (4) 调用外部函数

通过仅两行代码，就已经实现了根据输入（Prompt）正确匹配传入参数的功能。如果想实际调用该工具，只需将其传递给工具即可：

In [50]:
chain = llm_with_tools | JsonOutputKeyToolsParser(key_name='get_weather', first_tool_only=True) | get_weather
chain.invoke("北京现在的天气怎么样？")

'{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\\u9634\\uff0c\\u591a\\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 26.94, "feels_like": 26.27, "temp_min": 26.94, "temp_max": 26.94, "pressure": 1001, "humidity": 27, "sea_level": 1001, "grnd_level": 995}, "visibility": 10000, "wind": {"speed": 3.85, "deg": 222, "gust": 6.13}, "clouds": {"all": 94}, "dt": 1715317921, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715288663, "sunset": 1715339838}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}'

In [51]:
chain = llm_with_tools | JsonOutputKeyToolsParser(key_name='get_weather', first_tool_only=True) | get_weather
chain.invoke("今天杭州的天气好吗？")

'{"coord": {"lon": 120.1614, "lat": 30.2937}, "weather": [{"id": 802, "main": "Clouds", "description": "\\u591a\\u4e91", "icon": "03d"}], "base": "stations", "main": {"temp": 28.95, "feels_like": 29.48, "temp_min": 28.95, "temp_max": 28.95, "pressure": 1013, "humidity": 49, "sea_level": 1013, "grnd_level": 1011}, "visibility": 10000, "wind": {"speed": 4.39, "deg": 172, "gust": 5.89}, "clouds": {"all": 40}, "dt": 1715317358, "sys": {"type": 1, "id": 9651, "country": "CN", "sunrise": 1715288929, "sunset": 1715337765}, "timezone": 28800, "id": 1808926, "name": "Hangzhou", "cod": 200}'

通过这个流程，我们可以根据输入实时查询OpenWeather的API，并获取最终的查询结果。如果想进一步得到最终的回复，实现的逻辑应当是将返回的信息添加到Messages中，利用这些提示数据引导模型生成最终的回复。具体转化为代码的逻辑如下：

#### (5) 搭配Prompt Template完成外部函数调用

In [52]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "天气信息来源于OpenWeather API：https://api.openweathermap.org/data/2.5/weather"),
        ("system", "这是实时的天气数据：{weather_data}"),
        ("human", "{user_input}"),
    ]
)

In [54]:
chain = llm_with_tools | JsonOutputKeyToolsParser(key_name='get_weather', first_tool_only=True) | get_weather
weather_data = chain.invoke("今天杭州的天气好吗？")

In [55]:
weather_data

'{"coord": {"lon": 120.1614, "lat": 30.2937}, "weather": [{"id": 802, "main": "Clouds", "description": "\\u591a\\u4e91", "icon": "03d"}], "base": "stations", "main": {"temp": 29.95, "feels_like": 30.82, "temp_min": 29.95, "temp_max": 29.95, "pressure": 1013, "humidity": 49, "sea_level": 1013, "grnd_level": 1011}, "visibility": 10000, "wind": {"speed": 4.39, "deg": 172, "gust": 5.89}, "clouds": {"all": 40}, "dt": 1715318103, "sys": {"type": 1, "id": 9651, "country": "CN", "sunrise": 1715288929, "sunset": 1715337765}, "timezone": 28800, "id": 1808926, "name": "Hangzhou", "cod": 200}'

In [56]:
messages = chat_template.format_messages(weather_data=weather_data, user_input="今天杭州的天气好吗？")
messages

[SystemMessage(content='天气信息来源于OpenWeather API：https://api.openweathermap.org/data/2.5/weather'),
 SystemMessage(content='这是实时的天气数据：{"coord": {"lon": 120.1614, "lat": 30.2937}, "weather": [{"id": 802, "main": "Clouds", "description": "\\u591a\\u4e91", "icon": "03d"}], "base": "stations", "main": {"temp": 29.95, "feels_like": 30.82, "temp_min": 29.95, "temp_max": 29.95, "pressure": 1013, "humidity": 49, "sea_level": 1013, "grnd_level": 1011}, "visibility": 10000, "wind": {"speed": 4.39, "deg": 172, "gust": 5.89}, "clouds": {"all": 40}, "dt": 1715318103, "sys": {"type": 1, "id": 9651, "country": "CN", "sunrise": 1715288929, "sunset": 1715337765}, "timezone": 28800, "id": 1808926, "name": "Hangzhou", "cod": 200}'),
 HumanMessage(content='今天杭州的天气好吗？')]

#### (6) 生成最终答案

有了外部函数的执行结果，就可以把该结果（存储在聊天对话中、即messages变量）交给大模型，生成最终的答案了。
这里先手动调用大模型，后面可以通过自定义Output Parser编写更优雅的代码

In [57]:
response = chat.invoke(messages)
response

AIMessage(content='根据OpenWeather API提供的数据，今天杭州的天气为多云，温度约为29℃。', response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 317, 'total_tokens': 351}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-818a600e-96dd-44de-b0a2-76bf4ef34159-0')

In [58]:
print(response.content)

根据OpenWeather API提供的数据，今天杭州的天气为多云，温度约为29℃。


### 7.3 代码整理

至此，仅通过几行简单的代码，我们就已经快速实现了OpenAI的Function Calling功能，这得益于LangChain中预先抽象好的模块。接下来我们可以整理一下有效代码，如下：

#### (1) 构建外部函数

In [60]:
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

#### (2) 构建并使用Funcation Calling Chain

In [61]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 实例化大模型
openai_chat = ChatOpenAI(model_name="gpt-3.5-turbo",api_key=openai.api_key ,base_url=openai.api_base)

# 绑定外部工具
llm_with_tools = openai_chat.bind_tools([get_weather])

# 根据输入，调用指定的工具，并得到数据
chain = llm_with_tools | JsonOutputKeyToolsParser(key_name='get_weather', first_tool_only=True) | get_weather
weather_data = chain.invoke("今天杭州的天气好吗？")

# 构造输入模版，将工具返回的数据和当前的输入拼接到一起作为外部知识影响最终的输出
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "天气信息来源于OpenWeather API：https://api.openweathermap.org/data/2.5/weather"),
        ("system", "这是实时的天气数据：{weather_data}"),
        ("human", "{user_input}"),
    ]
)

# 生成messages
messages = chat_template.format_messages(weather_data=weather_data, user_input="今天杭州的天气好吗？")

# 实际进行推理
response = openai_chat.invoke(messages)
print(response.content)

根据OpenWeather API的数据，今天杭州的天气是多云的，温度为29.95摄氏度，湿度为49%。整体来说，天气还不错。


### 7.4 自定义输出解析器生成最终答案

#### (1) 问题和解决办法

前面的代码，前半部分比较优雅，但是不足之处是，仍然需要手动实现获取最终答案的部分。

```python
# 构造输入模版，将工具返回的数据和当前的输入拼接到一起作为外部知识影响最终的输出
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "天气信息来源于OpenWeather API：https://api.openweathermap.org/data/2.5/weather"),
        ("system", "这是实时的天气数据：{weather_data}"),
        ("human", "{user_input}"),
    ]
)

# 生成messages
messages = chat_template.format_messages(weather_data=weather_data, user_input="今天杭州的天气好吗？")

# 实际进行推理
response = openai_chat.invoke(messages)
print(response.content)
```

解决办法是实现一个自定义Output Parser，把这部分逻辑也整合到Chain中

#### (2) 自定义Output Parser要处理的输入数据

In [63]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 实例化大模型
openai_chat = ChatOpenAI(model_name="gpt-3.5-turbo",api_key=openai.api_key ,base_url=openai.api_base)

# 绑定外部工具
llm_with_tools = openai_chat.bind_tools([get_weather])

# 根据输入，调用指定的工具，并得到数据
chain = llm_with_tools | JsonOutputKeyToolsParser(key_name='get_weather', first_tool_only=True) | get_weather
weather_data = chain.invoke("今天北京的天气好吗？")
print(weather_data)

{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 804, "main": "Clouds", "description": "\u9634\uff0c\u591a\u4e91", "icon": "04d"}], "base": "stations", "main": {"temp": 26.94, "feels_like": 26.27, "temp_min": 26.94, "temp_max": 26.94, "pressure": 1001, "humidity": 27, "sea_level": 1001, "grnd_level": 995}, "visibility": 10000, "wind": {"speed": 3.85, "deg": 222, "gust": 6.13}, "clouds": {"all": 94}, "dt": 1715318552, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715288663, "sunset": 1715339838}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}


In [64]:
weather_data = chain.invoke("上海现在什么天气？")
print(weather_data)

{"coord": {"lon": 121.4581, "lat": 31.2222}, "weather": [{"id": 800, "main": "Clear", "description": "\u6674", "icon": "01d"}], "base": "stations", "main": {"temp": 26.61, "feels_like": 26.61, "temp_min": 24.93, "temp_max": 27.18, "pressure": 1015, "humidity": 51}, "visibility": 10000, "wind": {"speed": 6, "deg": 160}, "clouds": {"all": 0}, "dt": 1715318650, "sys": {"type": 2, "id": 145096, "country": "CN", "sunrise": 1715288518, "sunset": 1715337554}, "timezone": 28800, "id": 1796236, "name": "Shanghai", "cod": 200}


我们将要用到其中的“name”字段（城市），以及整个数据（天气信息）

#### (3) 编写自定义Output Parser

In [65]:
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import json

def final_resonse(ai_message: str) -> str:
    
    data = json.loads(ai_message)
   
    chat_template = ChatPromptTemplate.from_messages(
        [
            ("system", "这是实时的{city}的天气数据，信息来源于OpenWeather API：https://api.openweathermap.org/data/2.5/weather, 详细的数据是：{detail}",),
            ("system", "请你解析该数据，以自然语言的形式回复"),
        ]
    )
    
    # 生成messages
    messages = chat_template.format_messages(city=data["name"], detail=data)

    openai_chat = ChatOpenAI(model_name="gpt-3.5-turbo",api_key=openai.api_key ,base_url=openai.api_base)
    response = openai_chat.invoke(messages)
    return response.content

#### (4) 用自定义解析器简化代码

In [66]:
chain = llm_with_tools | JsonOutputKeyToolsParser(key_name='get_weather', first_tool_only=True) | get_weather | final_resonse
final_reponse = chain.invoke("北京现在的天气怎么样？")
final_reponse.replace('\n', '')

'当前北京的天气是阴天，多云。温度为26.94摄氏度，体感温度为26.27摄氏度。最低温度和最高温度都是26.94摄氏度。气压为1001hPa，湿度为27%。风速为3.85米/秒，风向为222度。云量为94%。能见度为10000米。'

In [67]:
chain = llm_with_tools | JsonOutputKeyToolsParser(key_name='get_weather', first_tool_only=True) | get_weather | final_resonse
final_reponse = chain.invoke("上海现在是什么天气状况？")
final_reponse.replace('\n', '')

'上海的天气信息如下：- 温度：26℃，体感温度也是26℃- 最低温度：24.93℃，最高温度：26.07℃- 气压：1015 hPa- 湿度：53%- 可见度：10000米- 风速：6 m/s，风向：160°- 天气状况：晴朗，没有云层- 数据更新时间：1715318818，当前时间的时间戳- 地理坐标：经度121.4581，纬度31.2222- 国家：中国，城市：上海- 日出时间：1715288518，日落时间：1715337554- 时区：+8小时- 数据来源：OpenWeather API希望以上信息对您有所帮助！'

## 8 LangChain调用开源模型的Funcation calling

### 8.1 代码编写

首先我们需要明确的是，OpenAI的GPT系列模型在很大程度上影响了大模型技术发展的开发范式和标准。所以无论是Qwen、ChatGLM等模型，它们的使用方法和函数调用逻辑基本遵循OpenAI定义的规范，没有太大差异。也正是这种一致性，现在大部分的开源项目才能够通过一个较为通用的接口来接入和使用不同的模型。这种兼容性和模型间的相似性之间存在直接联系。LangChain也不例外。

Ollama Functions：[https://python.langchain.com/docs/integrations/chat/ollama_functions](https://python.langchain.com/docs/integrations/chat/ollama_functions)

```python
    model = model.bind(
    functions=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)
```

In [129]:
! pip install langchain_experimental

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 193 kB 709 kB/s eta 0:00:01


In [143]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

In [144]:
model = OllamaFunctions(
    model="qwen:7b-chat"
)

In [145]:
@tool
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

In [146]:
from langchain_core.utils.function_calling import convert_to_openai_function

&emsp;&emsp;`convert_to_openai_function`的功能是将外部函数转化成Json Schema的表示，使用方法如下：

In [147]:
get_weather_json_schema = json.dumps(convert_to_openai_function(get_weather),ensure_ascii=False)
print(get_weather_json_schema)

{"name": "get_weather", "description": "get_weather(loc) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息", "parameters": {"type": "object", "properties": {"loc": {}}, "required": ["loc"]}}


In [148]:
print(type(get_weather_json_schema))

<class 'str'>


&emsp;&emsp;上述通过`json.dumps`是为了更好的显示输出，而实际上需要传入的Json Schema需要是字典形式，直接使用`convert_to_openai_function`方法进行转化。

In [149]:
get_weather_json_schema = convert_to_openai_function(get_weather)
print(get_weather_json_schema)

{'name': 'get_weather', 'description': "get_weather(loc) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息", 'parameters': {'type': 'object', 'properties': {'loc': {}}, 'required': ['loc']}}


In [82]:
print(type(get_weather_json_schema))

<class 'dict'>


In [150]:
functions_list = [get_weather_json_schema]
functions_list

[{'name': 'get_weather',
  'description': "get_weather(loc) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息",
  'parameters': {'type': 'object',
   'properties': {'loc': {}},
   'required': ['loc']}}]

In [151]:
model = model.bind(
    functions = functions_list,
    function_call={"name": "get_weather"},
)
model

RunnableBinding(bound=OllamaFunctions(llm=ChatOllama(model='qwen:7b-chat', format='json'), tool_system_prompt_template='You have access to the following tools:\n\n{tools}\n\nYou must always select one of the above tools and respond with only a JSON object matching the following schema:\n\n{{\n  "tool": <name of the selected tool>,\n  "tool_input": <parameters for the selected tool, matching the tool\'s JSON schema>\n}}\n'), kwargs={'functions': [{'name': 'get_weather', 'description': "get_weather(loc) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息", 'parameters': {'type': 'object', 'properties': {'loc': {}}, 'required': ['loc']}}], 'function_call': {'name': 'get_weather'}})

In [140]:
from langchain_core.messages import HumanMessage

model.invoke("查询一下北京的天气")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"loc": "\\u5317\\u4eac"}'}}, id='run-4bf84f61-c753-4621-ab70-f1e96bb62b7a-0')

&emsp;&emsp;将`get_weather`的Json Schma表示放在一个列表中。使用model.bind进行传入。

In [152]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [154]:
# 根据输入，调用指定的工具，并得到数据
chain = model | JsonKeyOutputFunctionsParser(key_name='loc', first_tool_only=True) | get_weather
weather_data = chain.invoke("上海现在什么天气？")
print(weather_data)

{"coord": {"lon": 121.4581, "lat": 31.2222}, "weather": [{"id": 800, "main": "Clear", "description": "\u6674", "icon": "01d"}], "base": "stations", "main": {"temp": 26.15, "feels_like": 26.15, "temp_min": 24.93, "temp_max": 26.92, "pressure": 1013, "humidity": 55}, "visibility": 10000, "wind": {"speed": 6, "deg": 160}, "clouds": {"all": 0}, "dt": 1715326560, "sys": {"type": 2, "id": 145096, "country": "CN", "sunrise": 1715288518, "sunset": 1715337554}, "timezone": 28800, "id": 1796236, "name": "Shanghai", "cod": 200}


到目前为止获取天气的数据了！

### 8.2 流程演示

In [164]:
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama


import json

def final_resonse(ai_message: str) -> str:
    
    data = json.loads(ai_message)
    print(data)
   
    chat_template = ChatPromptTemplate.from_messages(
        [
            ("system", "这是实时的{city}的天气数据，信息来源于OpenWeather API：https://api.openweathermap.org/data/2.5/weather, 详细的数据是：{detail}",),
            ("system", "请你解析该数据，以自然语言的形式回复"),
        ]
    )
    
    # 生成messages
    messages = chat_template.format_messages(city=data["name"], detail=data)

    # 实例化Ollama启动的模型
    ollama_llm = ChatOllama(model="qwen:7b-chat")
    response = ollama_llm.invoke(messages)
    return response.content

In [168]:
model = model.bind(
    functions = functions_list,
    function_call={"name": "get_weather"},
)

chain = model | JsonKeyOutputFunctionsParser(key_name='loc', first_tool_only=True) | get_weather | final_resonse
final_reponse = chain.invoke("上海现在什么天气？")
final_reponse.replace('\n', '')

{'coord': {'lon': 121.4581, 'lat': 31.2222}, 'weather': [{'id': 800, 'main': 'Clear', 'description': '晴', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 26.15, 'feels_like': 26.15, 'temp_min': 23.93, 'temp_max': 26.92, 'pressure': 1013, 'humidity': 57}, 'visibility': 10000, 'wind': {'speed': 6, 'deg': 160}, 'clouds': {'all': 0}, 'dt': 1715327333, 'sys': {'type': 2, 'id': 145096, 'country': 'CN', 'sunrise': 1715288518, 'sunset': 1715337554}, 'timezone': 28800, 'id': 1796236, 'name': 'Shanghai', 'cod': 200}


'这是一段关于上海实时天气的数据。具体信息如下：- **地理位置**：经度121.4581，纬度31.2222。- **天气状况**：晴朗（Weather Type: Clear, Description: Sunny）。- **温度**：当前气温为26.15°C，人体感觉温度与气温相近。- **湿度**：湿度为57%。- **气压**：气压为1013百帕。- **风速和方向**：风速为6公里/小时，风向为东偏南160°。- **其他时间信息**：日出时间为1715288518秒，日落时间为1715337554秒。上海的当前天气情况是晴朗，气温适中，湿度较大。如果你需要更详细的预报或者有其他关于天气的问题，请随时告诉我。'